# 电商PV、UV榜统计实战

In [1]:
pwd = !pwd
data_path = "file://" + list(pwd)[0] + '/data/'
print(data_path)

file:///root/pyspark-book/data/


In [2]:
rdd = sc.textFile(data_path + "access_log.txt")

In [3]:
rdd.first()

'105.145.171.151\t辽宁\t2021-06-01\t1532057592615\t4389195290417379165\twww.jd.com\tView'

In [4]:
# pv：网站的总访问量

In [5]:
def pv(rows):
    sitepair = rows.map(lambda row: (row.split("\t")[5], 1))
    result = sitepair.reduceByKey(lambda v1, v2: (v1+v2))
    
    # 按照降序排序
    result = result.sortBy(lambda one: one[1], ascending=False)
    
    return result

In [6]:
pv(rdd).collect()

[('www.suning.com', 9),
 ('www.baidu.com', 9),
 ('www.gome.com.cn', 7),
 ('www.taobao.com', 5),
 ('www.dangdang.com', 5),
 ('www.jd.com', 4),
 ('www.mi.com', 2)]

In [7]:
# uv：网站的独立用户访问量

In [8]:
def uv(rows):
    # 同一个ip，要distinct排重
    sitepair = rows\
        .map(lambda row: row.split("\t")[0] + "_" + row.split("\t")[5])\
        .distinct()

    result = sitepair.map(lambda row: (row.split("_")[1], 1))\
        .reduceByKey(lambda v1, v2: v1 + v2)\
        .sortBy(lambda kv: kv[1], ascending=False)    
    
    return result

In [9]:
uv(rdd).collect()

[('www.suning.com', 8),
 ('www.baidu.com', 8),
 ('www.gome.com.cn', 6),
 ('www.taobao.com', 5),
 ('www.jd.com', 4),
 ('www.dangdang.com', 3),
 ('www.mi.com', 2)]